# SVM: Máquinas de Vectores de Soporte

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

#### Preparar los datos

Las características se calculan a partir de una imagen digitalizada de un aspirado con aguja fina (FNA) de una masa mamaria. Describen características de los núcleos celulares presentes en la imagen en el espacio tridimensional es el descrito en: [K. P. Bennett y O. L. Mangasarian: "Discriminación de programación lineal robusta de dos conjuntos linealmente inseparables", Métodos de optimización y software 1, 1992, 23-34].

Variable Respuesta:
- Diagnosis (Maligno, Beningno)

Variables Independientes:

- a) radio (media de las distancias desde el centro hasta los puntos del perímetro)
- b) textura (desviación estándar de los valores de la escala de grises)
- c) perímetro
- d) área
- e) suavidad (variación local en longitudes de radio)
- f) compacidad (perímetro ^ 2 / área - 1.0)
- g) concavidad (severidad de las porciones cóncavas del contorno)
- h) puntos cóncavos (número de porciones cóncavas del contorno)
- i) simetría
- j) dimensión fractal ("aproximación de la línea costera" - 1)

Distribución de clases: 357 benignos, 212 malignos

In [2]:
# Dataset (Breast Cancer Wisconsin)
data = pd.read_csv("data/svm.csv")

# eliminamos la columna id y una columna NaN llamada Unnamed 32
data.drop(["id", "Unnamed: 32"], axis=1, inplace=True)

data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# Convertir la variable de Respuesta "diagnosis"
# Maligno: 1, no Maligno -1
tipos_diagnosis = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(tipos_diagnosis)
data['diagnosis'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: diagnosis, dtype: float64

In [4]:
# separamos las variables X-y
X = data.iloc[:, 1:]
y = data.loc[:, 'diagnosis']

In [5]:
# aplicar min-max scaler a X
X = pd.DataFrame(MinMaxScaler().fit_transform(X.values))

# insertar columna de unos para el intercept b0
X.insert(loc=len(X.columns), column='intercept', value=1)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,intercept
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,1
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,1
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,1
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,1
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,1


In [6]:
# vamos a crear el test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Máquinas de Vectores de Soporte

<img src="img/svm.jpeg">

SVM utiliza los datos del dataset para encontrar las vectores de sorpote que permitan la separación de las clases mediante un hiperplano. En el caso de SVM lineal, se utiliza la ecuación de la recta para separar las clases con $w^TX+b$ donde $w^TX+b < 0$ se clasifica como -1 o como 1 si $w^TX+b >= 0$

Los vectores de soporte son aquellos puntos del dataset que maximizan la separación de los datos mediante el hiperplano. Note que hablamos de hiperplano para +3D y de una simple recta para 1-2D, ya que el hiperplano (por ejemplo un plano con $A = b * h$) puede tener múltiples dimensiones. Vea la siguiente figura para ver un ejemplo de la segmentación con hiperplanos.

<img src="img/hiperplano.png">

En este caso el hiperplano, es capaz de separar datos en un ambiente de 3 dimensiones.

### Como de define el hiperplano?

En el SVM linea, el hiperplano esta dado por la ecuación de la recta $w^TX+b$ donde $b$ es el *intercept* y $w$ son los *coeficientes* (iguales a los coeficientes $\beta_i$ que utilizamos en Regresión Logistica), solo que aquí los llamamos $w$, al igual que en las redes neuronales.

La incognita es $w$, el cual vamos a estimar de los datos mediante Descenso de Gradiente donde los pesos $w$ óptimos se ecuentran cuando se minimiza la función del costo. 

$$f(x) = signo(w^TX+b)$$

- signo es **np.sign**. La función de signo devuelve -1 si x <0, 0 si x == 0, 1 si x> 0.

### Calculo del Costo

La funcion del costo en SVM es aquella que logra determinar donde encajar el hiperplano entre los vectores de sorporte. En la siguiente imagen, tenemos 3 hiperplanos ($H_1, H_2 y H_3$) que se pueden utilizar para separar los datos. 

<img src="img/h3.png" />

La siguiente fórmula define la función del costo $j(w)$:

$$ j(w) = \frac{1}{2}||w||^2 + C \left[  \frac{1}{N} \sum{max(0,1 - y_i * w^Tx_i+b)}  \right]$$

donde:
- w representa los coeficientes actuales
- $max(0,1 - y_i * w^Tx_i+b)$ control del margen (hinge loss).
- C es el termino de regularización. 

In [13]:
def costo_j(W, X, Y):
    
    N = X.shape[0]
    distancias = 1 - Y * (np.dot(X, W))
    distancias[distancias < 0] = 0  # max(0, distancias)
    hinge_loss = reg_C * (np.sum(distancias) / N)

    # calculo del costo
    costo = 1 / 2 * np.dot(W, W) + hinge_loss
    return costo

### Estimacion del Gradiente

$$ j(w) = \frac{1}{2}||w||^2 + C \left[  \frac{1}{N} \sum{max(0,1 - y_i * w^Tx_i+b)}  \right]$$

$$ \frac{d}{dw}j(w) = \frac{1}{N}\sum
    \begin{matrix}
      max(0,1 - y_i * w^Tx_i+b) = 0 & w\\
      else & w-Cy_ix_i
    \end{matrix}
$$

In [8]:
def gradiente(W, X_batch, Y_batch):
    
    # si solo se para 1 sample a la vez
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch]) 

    distancia = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distancia):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_C * Y_batch[ind] * X_batch[ind])
        dw += di

    # normalizado
    dw = dw/len(Y_batch)
    
    return dw

### Algoritmo de Descenso de Gradiente Estocástico

In [9]:
def DescensoGradiente(features, outputs):
    
    # maximo de iteraciones
    max_epochs = 5000 
    pesos = np.zeros(features.shape[1])
    nth = 0
    costo_anterior = float("inf")
    
    # criterio de convergencia
    conv_costo = 0.01
    
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        
        # Seleccion aleatoria de features
        # esto se llama Descenso de Gradiente Estocastico! 
        X, y = shuffle(features, outputs)
        
        # calculo de persos y gradientes
        for ind, x in enumerate(X):
            grad = gradiente(pesos, x, y[ind])
            pesos = pesos - (alfa * grad)

        # revision de convergencia
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            costo = costo_j(pesos, features, outputs)
            print("Epoch: {} Costo Actual: {}".format(epoch, costo))
            
            # detener si...
            if abs(costo_anterior - costo) < conv_costo * costo_anterior:
                return pesos
            
            costo_anterior = costo
            nth += 1
    
    return pesos

### Ejecutar entrenamiento: Calcular $w$

In [10]:
# definicion de hiperparametros
reg_C = 10000 # C regularizacion
alfa = 0.000001 # learning rate o tasa de aprendizaje

# entrenamiento de SVM con GD
W = DescensoGradiente(X_train.to_numpy(), y_train.to_numpy())
print("")
print("Persos w: {}".format(W))

Epoch: 1 Costo Actual: 5295.34665634401
Epoch: 2 Costo Actual: 3344.1680796587434
Epoch: 4 Costo Actual: 2429.1195502179794
Epoch: 8 Costo Actual: 1883.271739317526
Epoch: 16 Costo Actual: 1532.7515608045567
Epoch: 32 Costo Actual: 1222.3170806545136
Epoch: 64 Costo Actual: 961.4048065879952
Epoch: 128 Costo Actual: 803.8836617834102
Epoch: 256 Costo Actual: 713.8010614585663
Epoch: 512 Costo Actual: 666.3135915191677
Epoch: 1024 Costo Actual: 625.9598435746198
Epoch: 2048 Costo Actual: 620.38103926908

Persos w: [ 1.07626052  1.08026035  0.93929398  2.06146601 -1.14491755 -2.59574061
  3.35356077  5.88309881 -0.50573976 -0.42629918  5.01964235 -1.53134659
  3.0776419   3.23726098  1.66753423 -2.44096393 -1.18925315  0.26287951
 -1.62389032 -1.37512138  2.89350874  4.92558008  1.60988143  3.08971832
  2.39803088 -0.85539243  2.29640809  0.90793604  4.56074497  1.80101049
 -9.1936408 ]


### Calidad de la Prediccion

In [12]:
# prediccion del test set.
y_prima = np.sign(np.dot(X_test.to_numpy(), W))

accuracy_score(y_test, y_prima)

0.9736842105263158